In [1]:
# Pruning - Distillation Knowledge - Weight Sharing - Quantization all the Llama3 series models

In [1]:
from typing import List 
import fire 

import torch.distributed as dist 

from foxer import Llama 

In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

# Đường dẫn đến thư mục chứa mô hình
model_path = "/home/chwenjun225/.llama/checkpoints/Llama3.1-8B-Instruct/"

# Tải tokenizer
tokenizer = LlamaTokenizer.from_pretrained(model_path)

# Tải mô hình
model = LlamaForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=False,  # Nếu muốn dùng mô hình 8-bit để tiết kiệm VRAM
    torch_dtype=torch.float16,  # Tăng tốc độ bằng cách sử dụng float16 nếu có GPU
    device_map="auto"  # Tự động ánh xạ mô hình lên GPU nếu có
)

# Chuyển mô hình sang GPU nếu có
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Ví dụ prompt để thử nghiệm
prompt = "What is the capital of France?"

# Mã hóa prompt thành token
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Sinh văn bản
outputs = model.generate(inputs["input_ids"], max_length=50, temperature=0.7)

# Giải mã và in kết quả
print("Output:", tokenizer.decode(outputs[0], skip_special_tokens=True))


In [2]:
import torch
from transformers import LlamaForCausalLM

# Đường dẫn đến tệp consolidate.00.pth và nơi lưu pytorch_model.bin
input_path = "/home/chwenjun225/.llama/checkpoints/Llama3.1-8B-Instruct/consolidated.00.pth"
output_path = "/home/chwenjun225/.llama/checkpoints/Llama3.1-8B-Instruct/pytorch_model.bin"
config_path = "/home/chwenjun225/.llama/checkpoints/Llama3.1-8B-Instruct/params.json"

# Load trọng số từ consolidated.00.pth
state_dict = torch.load(input_path, map_location="cpu")

# Load kiến trúc mô hình từ params.json
model = LlamaForCausalLM.from_pretrained(pretrained_model_name_or_path="Llama3.1-8B-Instruct", state_dict=state_dict, config=config_path)

# Lưu trọng số thành định dạng pytorch_model.bin
model.save_pretrained("/home/chwenjun225/.llama/checkpoints/Llama3.1-8B-Instruct/bin/")

print(f"Model converted and saved at {output_path}")

/tmp/ipykernel_596958/1556114900.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(input_path, map_location="cpu")


OSError: Llama3.1-8B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

: 